In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
import requests
import re
import itertools
import time
from multiprocessing.pool import Pool
from multiprocessing import current_process
from functools import partial
import csv
import concurrent.futures
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'webdriver_manager'

# Main Functions

In [4]:
def get_cat(full_url):
    # Input: Page with cltohing types
    # Output: List of URLs of all types

    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15'
    #path = '/Users/anu/going_headless/chromedriver_mac64/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
    driver = webdriver.Chrome(executable_path =  ChromeDriverManager().install(), options=options)
    driver.get(full_url)
    
    elems = driver.find_elements_by_class_name("card__link-full")
    cat_list = [elem.get_attribute('href') for elem in elems]

    driver.quit()
    
    return cat_list

In [5]:
cat_list = get_cat('https://www.patagonia.com/shop/womens')

NameError: name 'ChromeDriverManager' is not defined

In [ ]:
cat_list.pop(0)
cat_list

In [ ]:
cat_dict = {}

In [ ]:
def cat_itemize(cat_url):
    # Input: URL of each category
    # Output: List of URLs of all items in each category
    
    global cat_dict

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15'
    path = '/Users/anu/going_headless/chromedriver_mac64/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
    driver = webdriver.Chrome(executable_path = path, options=options)
    driver.get(cat_url)
    
    # Name of the category
    cat_name = driver.find_element_by_xpath('/html/body/main/section/div[3]/div[1]/div/div[1]/div/div[1]/div').text

    if cat_name not in cat_dict:
        driver.get(cat_url)
        # If more than one page, click "LOAD MORE"
        while True:
            try:
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#product-search-results > div:nth-child(2) > div > div > div.row.product-grid.load-more-present > div.col-12.grid-footer > div.show-more.is-desktop-only > div > button")))
                button = driver.find_element_by_css_selector("#product-search-results > div:nth-child(2) > div > div > div.row.product-grid.load-more-present > div.col-12.grid-footer > div.show-more.is-desktop-only > div > button")
                button.location_once_scrolled_into_view
                button.click()
                time.sleep(5)
            except NoSuchElementException:
                break
            except TimeoutException:
                break

        all_items = driver.find_element_by_xpath("//*[@id='product-search-results']/div[2]/div/div/div[2]")
        items = all_items.find_elements_by_class_name("product-tile")
        cat_items = [item.get_attribute('data-monetate-producturl') for item in items]

        cat_dict[cat_name] = cat_items
        driver.quit()
    
    return cat_dict

In [ ]:
with open('patagonia_items.json', 'a') as bm:
    bm.write(json.dumps(cat_dict))

for i in range(len(cat_list)):
    cat_itemize(cat_list[i])

In [ ]:
with open('patagonia_items.json') as d:
    cat_dict_load = json.loads("[" + 
        d.read().replace("}{", "},\n{") + 
    "]")

In [ ]:
a = list(cat_dict_load[1].values())

a[0][0]

'https://www.patagonia.com/product/womens-organic-cotton-quilt-snap-t-pullover/25282.html'

In [ ]:
patag_url_list = []
for i in range(len(cat_dict_load)):
    l = list(cat_dict_load[i].values())
    for j in range(len(l[0])):
        patag_url_list.append(l[0][j])
len(patag_url_list)

770

In [ ]:
patag_url = patag_url_list[0]
patag_url

'https://www.patagonia.com/product/womens-torrentshell-3l-rain-jacket/85245.html'

In [ ]:
url_list = []

In [ ]:
def patag_scraper(patag_url_list, url_list):
  
    # Input: URL from Patagonia
    # Output: DF with results
    
    # Define empty lists to store results and log of failed URLs
    failed = []
    results = []
        
    for patag_url in tqdm(patag_url_list):
        if patag_url not in url_list:
            # Empty dictionary to store output
            patag_results = {}

            # Beautiful soup driver
            HEADERS = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4238.2  Safari/537.36',
            }
            content = requests.get(patag_url, headers=HEADERS)
            soup = BeautifulSoup(content.text, 'html.parser')

            # Selenium driver
            user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4238.2  Safari/537.36'
            path = '/Users/anu/going_headless/chromedriver_mac64/chromedriver'
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument("--window-size=1920,1080")
            options.add_argument('--ignore-certificate-errors')
            options.add_argument('--allow-running-insecure-content')
            options.add_argument(f'user-agent={user_agent}')
            options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
            driver = webdriver.Chrome(executable_path = path, options=options)
            driver.get(patag_url)

            # Check if product is in stock
            try:
#                 outofstock = driver.find_element_by_xpath('//*[@id="oos-label"]/h3').text
#                 if NoSuchElementException:
#                     pass
#                 if outofstock=='OUT OF STOCK':
#                     failed.append(patag_url)
#                     pass

#                 else:
                # Name of product
                patag_name = soup.find('div', {"class":"hero-pdp__intro-content"}).h1.get_text()
                patag_results['Name'] = patag_name

                # Material of product
                # *Click "Specs & Features" button*
                button = driver.find_element_by_xpath("/html/body/main/section/div[4]/section/div[2]/div/div/div/div/a")
                button.click()
                patag_mat = [el.text for el in driver.find_elements_by_xpath("/html/body/main/section/div[4]/section/div[4]/div/div/div/ul")]
                patag_results['Material'] = patag_mat

                # Color of product
                patag_color = []
                for color in soup.find('div', {"class":"row swiper-wrapper"}).find_all('h3',{'class':'product-tile__name'}):
                    patag_color.append(color.get_text())
                patag_results['Color'] = patag_color

                # URL of product
                patag_results['URL'] = patag_url

                # Image tag of product
                patag_image = []
                for img in soup.find('div', {"class":"row swiper-wrapper"}).find_all('div',{'data-image-zoom-url':True}):
                    patag_image.append(img['data-image-zoom-url'])        
                patag_results['Image'] = patag_image

                # Dict of Color : Image URL
                clr_img_dict = {}
                for color in patag_color:
                    for img in patag_image:
                        clr_img_dict[color] = img
                patag_results['Color:Image'] = clr_img_dict

                # Description of product
                patag_description = [desc.text for desc in driver.find_elements_by_xpath("/html/body/main/section/div[4]/section/div[3]/div/div/div/ul")]
                patag_results['Description'] = patag_description

                # Add dict output to list
                results.append(patag_results)

            except (NoSuchElementException, AttributeError):
                if NoSuchElementException:
                    failed.append(patag_url)
                elif AttributeError:
                    failed.append(patag_url)
                pass
    

            if len(results) > 0:            
                f = open("patag_table.csv", "a")
                writer = csv.DictWriter(
                    f, fieldnames=['Name', 'Material', 'Color', 'URL', 'Image', 'Color:Image', 'Description'])
                writer.writeheader()
                writer.writerows(results)
                f.close()
                results = []
            if len(failed) > 0:
                with open("failed_patag.txt", 'a') as ft:
                    for row in failed:
                        ft.write(str(row) + '\n')
                failed = []

# Scraper

In [ ]:
patag_urls = patag_url_list[156:]
print(len(patag_urls), patag_urls[0])


614 https://www.patagonia.com/product/womens-micro-d-snap-t-fleece-pullover/26020.html


In [ ]:
patag_url_list[570]

In [ ]:
patag_scraper(patag_url_list[571:], url_list)

100%|██████████| 199/199 [12:23<00:00,  3.73s/it]


In [ ]:
failed = open('failed_patag.txt','r').readlines()
len(failed)

225

In [ ]:
f = list(set(failed))
len(f)

95

In [ ]:
patag_df = pd.read_csv('patag_table.csv')
patag_df

,Name,Material,Color,URL,Image,Color:Image,Description
0,Women's Torrentshell 3L Jacket,"['H2No® Performance Standard shell: 3-layer, 3...","['Cornice Grey (CORG)', 'Catalan Coral (CCRL)'...",https://www.patagonia.com/product/womens-torre...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Cornice Grey (CORG)': 'https://www.patagonia...,['100% Recycled Waterproof/Breathable Face Fab...
1,Women's Nano Puff® Jacket,['Shell: 1.4-oz 20-denier 100% recycled polyes...,"['Birch White (BCW)', 'Black (BLK)', 'Feather ...",https://www.patagonia.com/product/womens-nano-...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Birch White (BCW)': 'https://www.patagonia.c...,['100% Recycled Shell Fabric\nLightweight 100%...
2,Women's Rainshadow Jacket,"['H2No® Performance Standard shell: 3-layer, 3...","['Classic Navy (CNY)', 'Roamer Red (RMRE)', 'C...",https://www.patagonia.com/product/womens-rains...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Classic Navy (CNY)': 'https://www.patagonia....,['Stretch 3-Layer H2No Performance Standard Sh...
3,Women's Calcite Jacket,"['2.5-layer, 3.4-oz 75-denier 100% recycled po...","['Feather Grey (FEA)', 'Classic Navy (CNY)', '...",https://www.patagonia.com/product/womens-calci...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Feather Grey (FEA)': 'https://www.patagonia....,['Lightweight and Packable\nLight and packable...
4,Women's Down Sweater Jacket,['Shell: 1.4-oz 20x30-denier 100% recycled pol...,"['Cornice Grey (CORG)', 'Black (BLK)', 'Birch ...",https://www.patagonia.com/product/womens-down-...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Cornice Grey (CORG)': 'https://www.patagonia...,['100% Recycled Shell Fabric\nDurable shell fa...
...,...,...,...,...,...,...,...
1063,Women's Insulated Prairie Dawn Parka,['Body: 8-oz 100% organic cotton canvas\nLinin...,"['Smolder Blue w/Smolder Blue (SMSB)', 'Sage K...",https://www.patagonia.com/product/womens-insul...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Smolder Blue w/Smolder Blue (SMSB)': 'https:...,['Fleece-Lined Organic Cotton Canvas/Thermogre...
1064,Name,Material,Color,URL,Image,Color:Image,Description
1065,Women's Kamala Cropped Pants,['5.1-oz 55% organic cotton/41% TENCEL™ lyocel...,"['Black (BLK)', 'Kale Green (KAGR)']",https://www.patagonia.com/product/womens-kamal...,['https://www.patagonia.com/dw/image/v2/BDJB_P...,{'Black (BLK)': 'https://www.patagonia.com/dw/...,['100% Organic Cotton/TENCEL™ Lyocell/Spandex ...
1066,Name,Material,Color,URL,Image,Color:Image,Description


In [ ]:
patag_df = patag_df.drop_duplicates()
patag_df.index = range(len(patag_df))
patag_df.to_csv('patag_table.csv', index=False)

In [ ]:
url_list = [i for i in patag_df['URL']] + f
len(url_list)

563

In [ ]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
t = Diff(patag_url_list, url_list)
len(t)

187

In [ ]:
patag_scraper(res, url_list)

100%|██████████| 191/191 [10:33<00:00,  3.32s/it]


In [ ]:
res = list(filter(None, t)) 
len(res)

191